### `tes-thermo`
---

In [3]:
from tes_thermo.thermo_agent import ThermoAgent
from tes_thermo.utils import VectorSearch
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os

In [4]:
load_dotenv(override=True)

# example using Azure OpenAI services
# Make sure to set the environment variables AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT,
# LLM_DEPLOYMENT_MODEL, EMBEDDING_DEPLOYMENT_MODEL, LLM_API_VERSION, and EMBEDDING_API_VERSION
# in your .env file or your environment before running this code.

class AzureServices():
    def __init__(self):
        self.AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
        self.AZURE_OPENAI_ENDPOINT= os.getenv("AZURE_OPENAI_ENDPOINT")
        self.DEPLOYMENT_MODEL = os.getenv("LLM_DEPLOYMENT_MODEL")
        self.EMBEGGIND_MODEL = os.getenv("EMBEDDING_DEPLOYMENT_MODEL")
        self.LLM_VERSION = os.getenv("LLM_API_VERSION")
        self.EMBEDDING_VERSION = os.getenv("EMBEDDING_API_VERSION")

    def get_default_llm(self):
        return AzureChatOpenAI(openai_api_version=self.LLM_VERSION,
                                azure_endpoint=self.AZURE_OPENAI_ENDPOINT,
                                openai_api_key=self.AZURE_OPENAI_API_KEY,
                                azure_deployment=self.DEPLOYMENT_MODEL,
                                temperature=0)
    
    def get_default_embedding(self):
        return AzureOpenAIEmbeddings(api_key=self.AZURE_OPENAI_API_KEY,
                                        api_version=self.EMBEDDING_VERSION,
                                        deployment=self.EMBEGGIND_MODEL,
                                        azure_endpoint=self.AZURE_OPENAI_ENDPOINT)

services = AzureServices()
llm = services.get_default_llm()
embedding_model = services.get_default_embedding()

In [5]:
vsearch = VectorSearch.from_documents(
    document_paths=['doc1...', 'doc2...'],  # replace with your document paths (pdf only...)
    embedding=embedding_model
)

File not found: 'catalysts-13-00884-v2.pdf'. Skipping.
No documents were processed. Empty index created.


In [ ]:
agent = ThermoAgent(llm = llm,
                    vsearch = vsearch,
                    embedding_model= embedding_model)

In [ ]:
prompt = """
Simulate the steam methane reforming (SMR) process. Consider methane, water, hydrogen, CO, and CO₂. 
The feed consists of 1 mol of water and 1 mol of CH₄ at 1200 K and 1 bar. 
Consult the literature on how to optimize the steam methane reforming process.
"""
res = agent.chat(prompt=prompt)
print(res['output'])